In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try: 
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.layers import Dense, Flatten, Dropout, Lambda, Permute, Reshape
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers.merge import Concatenate
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras import applications, losses, optimizers
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from tensorflow.python.framework import ops
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
from tensorflow.keras.callbacks import History
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
print(tf.__version__)

2.1.0


Using TensorFlow backend.


In [0]:
from scipy.spatial import distance
from PIL import Image
from keras.preprocessing import image
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import glob 
from numpy import genfromtxt
import pandas as pd

In [0]:
def load_data() -> Tuple[np.ndarray, np.ndarray, 
                          np.ndarray, np.ndarray]:
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train / 255.
    x_test = x_test / 255.
    return x_train, x_test, y_train, y_test

In [0]:
x_train, x_test, y_train, y_test = load_data()
print('x_train shape: {} | y_train shape: {}\nx_test shape : {} | y_test shape : {}'.format(x_train.shape, y_train.shape, 
x_test.shape, y_test.shape))
y_train_label = to_categorical(y_train, num_classes=10)
y_test_label = to_categorical(y_test, num_classes=10)

170500096/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3) | y_train shape: (50000, 1)
x_test shape : (10000, 32, 32, 3) | y_test shape : (10000, 1)


In [0]:
def grayscale(data, dtype='float32'):
    # luma coding weighted average in video systems
    r, g, b = np.asarray(.3, dtype=dtype), np.asarray(.59, dtype=dtype), np.asarray(.11, dtype=dtype)
    rst = r * data[:, :, :, 0] + g * data[:, :, :, 1] + b * data[:, :, :, 2]
    # add channel dimension
    rst = np.expand_dims(rst, axis=3)
    return rst



In [0]:
x_train_gray_images = grayscale(x_train)
x_test_gray_images = grayscale(x_test)
x_train_red_images = np.expand_dims(x_train[:, :, :, 0], axis = 3)
x_test_red_images = np.expand_dims(x_test[:, :, :, 0], axis = 3)
x_train_green_images = np.expand_dims(x_train[:, :, :, 1], axis = 3)
x_test_green_images = np.expand_dims(x_test[:, :, :, 1], axis = 3)
x_train_blue_images = np.expand_dims(x_train[:, :, :, 2], axis = 3)
x_test_blue_images = np.expand_dims(x_test[:, :, :, 2], axis = 3)

In [0]:
input_shape = x_train[0,:,:,:].shape
gray_input_shape = x_train_gray_images[0,:,:,:].shape
red_input_shape = x_train_red_images[0,:,:,:].shape
blue_input_shape = x_train_blue_images[0,:,:,:].shape
green_input_shape = x_train_green_images[0,:,:,:].shape
model_input = tf.keras.layers.Input(shape=input_shape)
model_gray_input = tf.keras.layers.Input(shape=gray_input_shape)
model_red_input = tf.keras.layers.Input(shape=red_input_shape)
model_green_input = tf.keras.layers.Input(shape=green_input_shape)
model_blue_input = tf.keras.layers.Input(shape=blue_input_shape)

In [0]:
def compile_with_train_generator(model: training.Model, num_epochs: int, batch_size: int, x_train, x_test) -> Tuple [History, str]:
  #y_train_label = to_categorical(y_train)
  #y_test_label = to_categorical(y_test) #one hot encoding
  model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
  # path2 = os.path.join(PATH, 'final_model2_v1.h5')
  # create data generator
  datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True) #, zoom_range = [0.9, 1.1], shear_range = 0.1
  # prepare iterator
  it_train = datagen.flow(x_train, y_train_label, batch_size=batch_size)
  # fit model
  steps = int(x_train.shape[0] / batch_size)
  history = model.fit_generator(it_train, steps_per_epoch=steps, epochs= num_epochs, validation_data=(x_test, y_test_label), verbose=1)
  
  weight_files = "" #glob.glob(os.path.join(os.getcwd(), 'weights/*'))
  weight_file = "model" #max(weight_files, key=os.path.getctime) # most recent file
  return history, weight_file

In [0]:
def all_stack_cnn_filter_size4_strid2(model_input: Tensor) -> training.Model:
    x1 = Conv2D(32, kernel_size=(4, 4), activation='relu', padding = 'same')(model_input)
    x1 = Conv2D(32, (4, 4), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x1)
    x1 = Dropout(0.2) (x1)
    x1 = BatchNormalization()(x1)
    x1 = Conv2D(32, (4, 4), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x1)

    x1 = Conv2D(64, (4, 4), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x1)
    x1 = Conv2D(64, (4, 4), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x1)
    x1 = Dropout(0.3) (x1)
    x1 = BatchNormalization()(x1)
    x1 = Conv2D(64, (4, 4), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x1)

    x = Conv2D(64, (4, 4), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x1)
    x = Dropout(0.4) (x)
    x = BatchNormalization()(x)
    
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)

    model = Model(model_input, x, name='all_stack_cnn_filter_size4_strid2')
    
    return model

In [0]:
batch_size = 128
num_epochs = 50
all_stack_cnn_filter_size4_strid2_gray_model = all_stack_cnn_filter_size4_strid2(model_gray_input)
_, conv_pool_cnn_weight_file = compile_with_train_generator(all_stack_cnn_filter_size4_strid2_gray_model, num_epochs, batch_size, 
                                                            x_train_gray_images, x_test_gray_images)

  ...
    to  
  ['...']
Train for 390 steps, validate on 10000 samples
Epoch 1/50
390/390 [==============================] - 17s 42ms/step - loss: 1.6158 - accuracy: 0.4162 - val_loss: 5.4732 - val_accuracy: 0.1709
Epoch 2/50
390/390 [==============================] - 15s 39ms/step - loss: 1.2005 - accuracy: 0.5751 - val_loss: 1.4055 - val_accuracy: 0.5222
Epoch 3/50
390/390 [==============================] - 15s 39ms/step - loss: 1.0332 - accuracy: 0.6367 - val_loss: 1.5020 - val_accuracy: 0.5063
Epoch 4/50
390/390 [==============================] - 15s 39ms/step - loss: 0.9296 - accuracy: 0.6762 - val_loss: 1.1657 - val_accuracy: 0.6196
Epoch 5/50
390/390 [==============================] - 15s 39ms/step - loss: 0.8539 - accuracy: 0.7029 - val_loss: 0.8845 - val_accuracy: 0.7014
Epoch 6/50
390/390 [==============================] - 15s 37ms/step - loss: 0.7990 - accuracy: 0.7228 - val_loss: 0.8837 - val_accuracy: 0.7062
Epoch 7/50
390/390 [==============================] - 14s 37ms/s

In [0]:
batch_size = 128
num_epochs = 50
all_stack_cnn_filter_size4_strid2_red_model = all_stack_cnn_filter_size4_strid2(model_red_input)
_, conv_pool_cnn_weight_file = compile_with_train_generator(all_stack_cnn_filter_size4_strid2_red_model, num_epochs, batch_size, 
                                                            x_train_red_images, x_test_red_images)

  ...
    to  
  ['...']
Train for 390 steps, validate on 10000 samples
Epoch 1/50
390/390 [==============================] - 15s 38ms/step - loss: 1.5945 - accuracy: 0.4220 - val_loss: 5.8864 - val_accuracy: 0.1037
Epoch 2/50
390/390 [==============================] - 14s 35ms/step - loss: 1.2003 - accuracy: 0.5762 - val_loss: 1.1361 - val_accuracy: 0.6088
Epoch 3/50
390/390 [==============================] - 14s 36ms/step - loss: 1.0316 - accuracy: 0.6386 - val_loss: 1.0690 - val_accuracy: 0.6236
Epoch 4/50
390/390 [==============================] - 14s 35ms/step - loss: 0.9260 - accuracy: 0.6757 - val_loss: 1.1724 - val_accuracy: 0.6213
Epoch 5/50
390/390 [==============================] - 14s 35ms/step - loss: 0.8507 - accuracy: 0.7043 - val_loss: 0.8479 - val_accuracy: 0.7131
Epoch 6/50
390/390 [==============================] - 14s 36ms/step - loss: 0.8027 - accuracy: 0.7215 - val_loss: 0.7880 - val_accuracy: 0.7306
Epoch 7/50
390/390 [==============================] - 14s 36ms/s

In [0]:
batch_size = 128
num_epochs = 50
all_stack_cnn_filter_size4_strid2_green_model = all_stack_cnn_filter_size4_strid2(model_green_input)
_, conv_pool_cnn_weight_file = compile_with_train_generator(all_stack_cnn_filter_size4_strid2_green_model, num_epochs, batch_size, 
                                                            x_train_green_images, x_test_green_images)

  ...
    to  
  ['...']
Train for 390 steps, validate on 10000 samples
Epoch 1/50
390/390 [==============================] - 14s 37ms/step - loss: 1.6168 - accuracy: 0.4111 - val_loss: 10.3830 - val_accuracy: 0.1000
Epoch 2/50
390/390 [==============================] - 14s 35ms/step - loss: 1.2075 - accuracy: 0.5729 - val_loss: 1.3351 - val_accuracy: 0.5340
Epoch 3/50
390/390 [==============================] - 14s 35ms/step - loss: 1.0278 - accuracy: 0.6398 - val_loss: 1.1912 - val_accuracy: 0.5888
Epoch 4/50
390/390 [==============================] - 14s 35ms/step - loss: 0.9263 - accuracy: 0.6769 - val_loss: 0.8863 - val_accuracy: 0.6923
Epoch 5/50
390/390 [==============================] - 14s 35ms/step - loss: 0.8512 - accuracy: 0.7043 - val_loss: 1.6163 - val_accuracy: 0.5631
Epoch 6/50
390/390 [==============================] - 14s 35ms/step - loss: 0.7950 - accuracy: 0.7262 - val_loss: 0.9530 - val_accuracy: 0.6864
Epoch 7/50
390/390 [==============================] - 13s 35ms/

In [0]:
batch_size = 128
num_epochs = 50
all_stack_cnn_filter_size4_strid2_blue_model = all_stack_cnn_filter_size4_strid2(model_blue_input)
_, conv_pool_cnn_weight_file = compile_with_train_generator(all_stack_cnn_filter_size4_strid2_blue_model, num_epochs, batch_size, 
                                                            x_train_gray_images, x_test_blue_images)

  ...
    to  
  ['...']
Train for 390 steps, validate on 10000 samples
Epoch 1/50
390/390 [==============================] - 14s 36ms/step - loss: 1.6194 - accuracy: 0.4103 - val_loss: 3.6453 - val_accuracy: 0.1561
Epoch 2/50
390/390 [==============================] - 14s 35ms/step - loss: 1.2234 - accuracy: 0.5646 - val_loss: 1.3158 - val_accuracy: 0.5375
Epoch 3/50
390/390 [==============================] - 14s 35ms/step - loss: 1.0593 - accuracy: 0.6262 - val_loss: 1.1494 - val_accuracy: 0.6131
Epoch 4/50
390/390 [==============================] - 13s 34ms/step - loss: 0.9569 - accuracy: 0.6632 - val_loss: 1.1359 - val_accuracy: 0.6246
Epoch 5/50
390/390 [==============================] - 14s 35ms/step - loss: 0.8807 - accuracy: 0.6909 - val_loss: 1.1160 - val_accuracy: 0.6375
Epoch 6/50
390/390 [==============================] - 14s 35ms/step - loss: 0.8352 - accuracy: 0.7107 - val_loss: 1.0568 - val_accuracy: 0.6548
Epoch 7/50
390/390 [==============================] - 14s 35ms/s

In [0]:
def generate_generator_multiple(generator, image_data, batch_size, img_height,img_width):
    genX1 = generator.flow(image_data[0],
                            target_size = (img_height,img_width),
                            class_mode = 'categorical',
                            batch_size = batch_size,
                            shuffle=False, 
                            seed=7)
    
    genX2 = generator.flow_from_directory(image_data[1],
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=7)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            yield [X1i[0], X2i[0]], X2i[1]  #Yield both images and their mutual label
            
            
inputgenerator=generate_generator_multiple(generator=input_imgen,
                                           dir1=train_dir_1,
                                           dir2=train_dir_2,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_height)       
     
testgenerator=generate_generator_multiple(test_imgen,
                                          dir1=train_dir_1,
                                          dir2=train_dir_2,
                                          batch_size=batch_size,
                                          img_height=img_height,
                                          img_width=img_height)              
          
 history=model.fit_generator(inputgenerator,
                        steps_per_epoch=trainsetsize/batch_size,
                        epochs = epochs,
                        validation_data = testgenerator,
                        validation_steps = testsetsize/batch_size,
                        use_multiprocessing=True,
                        shuffle=False)

IndentationError: ignored

In [0]:
def generate_generator_multiple_inputs(generator, image_data, image_lable, batch_size):
    genX1 = generator.flow(image_data[0], image_lable, batch_size = batch_size)
    #genX2 = generator.flow(image_data[0], image_data[1], batch_size = batch_size)
    genX2 = generator.flow(image_data[1], image_lable, batch_size = batch_size)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            yield [X1i[0], X2i[0]], X1i[1]  #Yield both images and their mutual label
            #yield [X1i[0], X2i[1]], X1i[1]  #Yield both images and their mutual label

In [0]:
def compile_with_train_generator_with_multi_input(model: training.Model, num_epochs: int, batch_size: int, x_train, x_test) -> Tuple [History, str]:
  #y_train_label = to_categorical(y_train)
  #y_test_label = to_categorical(y_test) #one hot encoding
  model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
  # path2 = os.path.join(PATH, 'final_model2_v1.h5')
  # create data generator
  # Finally create generator
 
  train_datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True) #, zoom_range = [0.9, 1.1], shear_range = 0.1
  test_datagen = ImageDataGenerator()

  # prepare iterator
  train_input_it = generate_generator_multiple_inputs(train_datagen, x_train, y_train_label, batch_size)
  test_input_it = generate_generator_multiple_inputs(test_datagen, x_test, y_test_label, batch_size)

  # fit model
  steps = int(x_train[0].shape[0] / batch_size)
  #history = model.fit_generator(train_input_it, steps_per_epoch=steps, epochs= num_epochs, validation_data = ([x_test[0], x_test[1]], y_test_label), verbose=1)
  history = model.fit_generator(train_input_it, steps_per_epoch=steps, epochs= num_epochs, validation_data = test_input_it, validation_steps = steps, verbose=1)
  weight_files = "" #glob.glob(os.path.join(os.getcwd(), 'weights/*'))
  weight_file = "model" #max(weight_files, key=os.path.getctime) # most recent file
  return history, weight_file

In [0]:
def multi_color_merge_stack_cnn_diff_filter_size_strid2(model_input: Tensor) -> training.Model:
    x1 = Conv2D(64, kernel_size=(4, 4), activation='relu', padding = 'same')(model_input[0])
    x1 = Conv2D(64, (4, 4), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x1)
    x1 = Dropout(0.25) (x1)
    x1 = BatchNormalization()(x1)
    x1 = Conv2D(64, (4, 4), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x1)

    x2 = Conv2D(64, kernel_size=(4, 4), activation='relu', padding = 'same')(model_input[1])
    x2 = Conv2D(64, (3, 3), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x2)
    x2 = Dropout(0.25) (x2)
    x2 = BatchNormalization()(x2)
    x2 = Conv2D(64, (3, 3), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x2)

    # x3 = Conv2D(64, kernel_size=(4, 4), activation='relu', padding = 'same')(model_input[2])
    # x3 = Conv2D(64, (3, 3), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x3)
    # x3 = Dropout(0.25) (x3)
    # x3 = BatchNormalization()(x3)
    # x3 = Conv2D(64, (3, 3), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x3)

    # x4 = Conv2D(64, kernel_size=(4, 4), activation='relu', padding = 'same')(model_input[3])
    # x4 = Conv2D(64, (3, 3), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x4)
    # x4 = Dropout(0.25) (x4)
    # x4 = BatchNormalization()(x4)
    # x4 = Conv2D(64, (3, 3), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x4)

    x = layers.Add()([x1, x2])
    x = Conv2D(64, (2, 2), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)

    model = Model(model_input, x, name='multi_color_merge_stack_cnn_diff_filter_size_strid2')
    
    return model

In [0]:
batch_size = 128
num_epochs = 50
model_multi_input = [model_gray_input, model_red_input] #, model_green_input, model_blue_input]
x_train_multi_images = [x_train_gray_images, x_train_red_images]
x_test_multi_images = [x_test_gray_images, x_test_red_images]
multi_color_merge_stack_cnn_diff_filter_size_strid2_model = multi_color_merge_stack_cnn_diff_filter_size_strid2(model_multi_input)
_, conv_pool_cnn_weight_file = compile_with_train_generator_with_multi_input(multi_color_merge_stack_cnn_diff_filter_size_strid2_model, 
                                                                             num_epochs, batch_size, x_train_multi_images, x_test_multi_images)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 390 steps, validate for 390 steps
Epoch 1/50
390/390 [==============================] - 32s 82ms/step - loss: 1.8226 - accuracy: 0.3300 - val_loss: 1.8679 - val_accuracy: 0.3510
Epoch 2/50
390/390 [==============================] - 31s 79ms/step - loss: 1.5520 - accuracy: 0.4454 - val_loss: 1.5172 - val_accuracy: 0.4640
Epoch 3/50
390/390 [==============================] - 31s 80ms/step - loss: 1.4337 - accuracy: 0.4922 - val_loss: 1.8174 - val_accuracy: 0.4034
Epoch 4/50
390/390 [==============================] - 31s 80ms/step - loss: 1.3478 - accuracy: 0.5242 - val_loss: 1.5322 - val_accuracy: 0.4829
Epoch 5/50
390/390 [==============================] - 31s 80ms/step - loss: 1.2766 - accuracy: 0.5533 - val_loss: 1.3442 - val_accuracy: 0.5381
Epoch 6/50
390/390 [==============================] - 31s 80ms/step - loss: 1.2320 - accuracy: 0.5652 - val_loss: 1.1935 - val_accuracy: 0.5693
Epoch 7/50
390/390 [======================

In [0]:
def multi_color_merge_stack_cnn_diff_filter_size_strid2_avg(model_input: Tensor) -> training.Model:
    x1 = Conv2D(64, kernel_size=(4, 4), activation='relu', padding = 'same')(model_input[0])
    x1 = Conv2D(64, (4, 4), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x1)
    x1 = Dropout(0.25) (x1)
    x1 = BatchNormalization()(x1)
    x1 = Conv2D(64, (4, 4), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x1)

    x2 = Conv2D(64, kernel_size=(4, 4), activation='relu', padding = 'same')(model_input[1])
    x2 = Conv2D(64, (3, 3), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x2)
    x2 = Dropout(0.25) (x2)
    x2 = BatchNormalization()(x2)
    x2 = Conv2D(64, (3, 3), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x2)

    #x = layers.Add()([x1, x2])
    x = layers.average([x1, x2])
    x = Conv2D(64, (2, 2), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)

    model = Model(model_input, x, name='multi_color_merge_stack_cnn_diff_filter_size_strid2_avg')
    return model

In [0]:
batch_size = 128
num_epochs = 50
model_multi_input = [model_gray_input, model_red_input] #, model_green_input, model_blue_input]
x_train_multi_images = [x_train_gray_images, x_train_red_images]
x_test_multi_images = [x_test_gray_images, x_test_red_images]
multi_color_merge_stack_cnn_diff_filter_size_strid2_avg_model = multi_color_merge_stack_cnn_diff_filter_size_strid2_avg(model_multi_input)
_, conv_pool_cnn_weight_file = compile_with_train_generator_with_multi_input(multi_color_merge_stack_cnn_diff_filter_size_strid2_avg_model, 
                                                                             num_epochs, batch_size, x_train_multi_images, x_test_multi_images)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 390 steps, validate for 390 steps
Epoch 1/50
390/390 [==============================] - 35s 89ms/step - loss: 1.8276 - accuracy: 0.3299 - val_loss: 1.8175 - val_accuracy: 0.4002
Epoch 2/50
390/390 [==============================] - 34s 87ms/step - loss: 1.5374 - accuracy: 0.4469 - val_loss: 2.4057 - val_accuracy: 0.3196
Epoch 3/50
390/390 [==============================] - 34s 86ms/step - loss: 1.4088 - accuracy: 0.4972 - val_loss: 1.4355 - val_accuracy: 0.4992
Epoch 4/50
390/390 [==============================] - 34s 87ms/step - loss: 1.3394 - accuracy: 0.5273 - val_loss: 1.5050 - val_accuracy: 0.4925
Epoch 5/50
390/390 [==============================] - 34s 86ms/step - loss: 1.2708 - accuracy: 0.5535 - val_loss: 1.2471 - val_accuracy: 0.5629
Epoch 6/50
390/390 [==============================] - 33s 85ms/step - loss: 1.2257 - accuracy: 0.5673 - val_loss: 1.2972 - val_accuracy: 0.5388
Epoch 7/50
390/390 [======================

In [0]:
def multi_color_merge_stack_cnn_diff_filter_size_strid2_max(model_input: Tensor) -> training.Model:
    x1 = Conv2D(64, kernel_size=(4, 4), activation='relu', padding = 'same')(model_input[0])
    x1 = Conv2D(64, (4, 4), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x1)
    x1 = Dropout(0.25) (x1)
    x1 = BatchNormalization()(x1)
    x1 = Conv2D(64, (4, 4), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x1)

    x2 = Conv2D(64, kernel_size=(4, 4), activation='relu', padding = 'same')(model_input[1])
    x2 = Conv2D(64, (3, 3), activation='relu', padding = 'same', kernel_initializer='he_uniform')(x2)
    x2 = Dropout(0.25) (x2)
    x2 = BatchNormalization()(x2)
    x2 = Conv2D(64, (3, 3), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x2)

    #x = layers.Add()([x1, x2])
    x = layers.maximum([x1, x2])
    x = Conv2D(64, (2, 2), activation='relu', padding = 'same', strides = 2, kernel_initializer='he_uniform')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)

    model = Model(model_input, x, name='multi_color_merge_stack_cnn_diff_filter_size_strid2_max')
    return model

In [0]:
batch_size = 128
num_epochs = 50
model_multi_input = [model_gray_input, model_red_input] #, model_green_input, model_blue_input]
x_train_multi_images = [x_train_gray_images, x_train_red_images]
x_test_multi_images = [x_test_gray_images, x_test_red_images]
multi_color_merge_stack_cnn_diff_filter_size_strid2_max_model = multi_color_merge_stack_cnn_diff_filter_size_strid2_max(model_multi_input)
_, conv_pool_cnn_weight_file = compile_with_train_generator_with_multi_input(multi_color_merge_stack_cnn_diff_filter_size_strid2_max_model, 
                                                                             num_epochs, batch_size, x_train_multi_images, x_test_multi_images)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 390 steps, validate for 390 steps
Epoch 1/50
390/390 [==============================] - 33s 85ms/step - loss: 1.8186 - accuracy: 0.3350 - val_loss: 1.8562 - val_accuracy: 0.3985
Epoch 2/50
390/390 [==============================] - 32s 81ms/step - loss: 1.5400 - accuracy: 0.4486 - val_loss: 1.5135 - val_accuracy: 0.4565
Epoch 3/50
390/390 [==============================] - 31s 81ms/step - loss: 1.4224 - accuracy: 0.4946 - val_loss: 1.7644 - val_accuracy: 0.4059
Epoch 4/50
390/390 [==============================] - 31s 80ms/step - loss: 1.3465 - accuracy: 0.5232 - val_loss: 1.2628 - val_accuracy: 0.5578
Epoch 5/50
390/390 [==============================] - 31s 80ms/step - loss: 1.2790 - accuracy: 0.5488 - val_loss: 1.6540 - val_accuracy: 0.4490
Epoch 6/50
390/390 [==============================] - 31s 80ms/step - loss: 1.2344 - accuracy: 0.5655 - val_loss: 1.4641 - val_accuracy: 0.4849
Epoch 7/50
390/390 [======================